In [ ]:

import os
from dotenv import load_dotenv
from openai import OpenAI
from IPython.display import Markdown, display
import requests
import gradio as gr

In [ ]:
load_dotenv(override=True)
openai_api_key = os.getenv('OPENAI_API_KEY')
google_api_key = os.getenv('GEMINI_API_KEY')
groq_api_key = os.getenv('GROQ_API_KEY')
openrouter_api_key = os.getenv('OPENROUTER_API_KEY')

In [ ]:
# constants
gemini_url = "https://generativelanguage.googleapis.com/v1beta/openai/"
groq_url = "https://api.groq.com/openai/v1"
openrouter_url = "https://openrouter.ai/api/v1"
ollama_url = "http://localhost:11434/v1"

In [ ]:
# Connect to OpenAI client library
openai = OpenAI()
gemini = OpenAI(api_key=google_api_key, base_url=gemini_url)
groq = OpenAI(api_key=groq_api_key, base_url=groq_url)
openrouter = OpenAI(base_url=openrouter_url, api_key=openrouter_api_key)
ollama = OpenAI(api_key="ollama", base_url=ollama_url)

In [ ]:
system_message = "you are a helpful assistant"

def message_gpt(prompt):
    messages = [
        {"role": "system", "content": system_message},
        {"role": "user", "content": prompt}
    ]
    response = openai.chat.completions.create(model="gpt-4.1-mini", messages=messages)

    return response.choices[0].message.content

In [ ]:
message_gpt("what is today's date?")

In [ ]:
def printData(data):
    print(f"printing the value {data}")
    return data.upper()

In [ ]:
print(printData("hello"))


In [ ]:
gr.Interface(fn=printData, inputs="textbox", outputs="textbox", flagging_mode="never").launch(inbrowser=True, auth=("riju", "pass123"))

In [ ]:
message_input = gr.Textbox(label="Your message:", info="Enter a message for GPT-4.1-mini", lines=7)
message_output = gr.Textbox(label="Response:", lines=7)

view = gr.Interface(
    fn=message_gpt,
    title="GPT-4.1",
    inputs=[message_input],
    outputs=[message_output],
    examples=["hello", "howdy"],
    flagging_mode="never"
)

view.launch()

In [ ]:
system_message = "You are a helpful assistant who responds in Markdown without code blocks"

message_input = gr.Textbox(label="Your message:", info="Enter a message for GPT-4.1-mini", lines=7)
message_output = gr.Markdown(label="Response:")


view = gr.Interface(
    fn=message_gpt,
    title="GPT-4.1",
    inputs=[message_input],
    outputs=[message_output],
    examples=[
        "Explain the transformer architecture to a layperson", 
        "Explain the transformer architecture to an aspriting AI person"
        ],
    flagging_mode="never"
)

view.launch()

In [ ]:
def chat(message, history):
    return f"history {history}"


In [ ]:
gr.ChatInterface(fn=chat).launch()

In [ ]:
def updated_chat_with_llms(message, history):
    print("asking llm...")
    history=[ {"role": h["role"], "content": h["content"]} for h in history]
    messages = [{"role": "system", "content": "you are a highly useful assistant"}] + history + [{"role": "user", "content": message}]

    response = openai.chat.completions.create(
        model="gpt-4.1-mini",
        messages=messages
    )

    return response.choices[0].message.content

In [ ]:
gr.ChatInterface(fn=updated_chat_with_llms).launch()

In [ ]:
def chat_as_stream(message, history):
    print(f"incoming message: {message}. asking the llm...")
    history = [{"role": h["role"], "content": h["content"]} for h in history]
    messages = [{"role": "system", "content": "You are a very helpful assistant"}] + history + [{"role": "user", "content": message}]